In [51]:
import pandas as pd
import numpy as np
import os
import math
from scipy.optimize import fsolve

In [31]:
# Significant Wave Height [unit: meter]
Hm0 = 1.14

# Peak Wave Period (Associated) [unit: seconds]
Tp = 3.36*math.sqrt(Hm0)

# Slope of front of structure (1:n)
n = 5

# Berm width [unit: meter]
B = 0

# Berm length [unit: meter]
Lberm = 1

# Depth at berm [unit: meter]
db = 0

# Estimate of Ru2% [unit: meter]
Ru2_estimate = 4.2

# Slope roughness factor
γf = 1

# Angle of attack [unit: degree]
β = 20

# Vertical wall factor
γv = 1

# Gravitational acceleration
g = 9.81

# Freeboard [unit: degree]
RC = 0.90

In [49]:
# Slope of front of structure {tan(α) = 1/n}
tanα = 1/n

# Spectral wave period {Tm-1,0 = Tp / 1.1}
Tm_1_0 = Tp / 1.1

# Deep water wave length {Lm-1,0=g*Tm-1,0^2/(2*PI)}
Lm_1_0 = (g * Tm_1_0**2) /(2 * math.pi)

# Wave steepness {Hm0/Lm-1,0}
Sm_1_0 = Hm0 / Lm_1_0

# Breaker parameter, Xi {ξm-1,0 = tan(α)/(sm-1,0)**0.5}
# Surf similarity or Iribarren number. Xi<2-3 for breaking waves
ξm_1_0 = tanα / (Sm_1_0**0.5)

# Berm reduction, first part {rB = B/Lberm}
rB = B / Lberm


# Berm reduction, second part
# {db>0: 0.5-0.5*cos(pi*db/Ru2%)  db<0: 0.5-0.5*cos(pi*db/(2*hm0)). Always >=0.6}

if db>0:
    rdb_cal = 0.5 - 0.5 * math.cos(math.pi * db / (Ru2_estimate) )
    rdb = max(0.6, rdb_cal)

elif db<0:
    rdb_cal = 0.5 - 0.5 * math.cos(math.pi * db / (2 * Hm0) )
    rdb = max(0.6, rdb_cal)

else:
    rdb = 0.6

    
# Berm influence factor (Between 0.6 and 1.0)
γb = max(1, min(0.6, 1 - rB * (1 - rdb)))


# Oblique wave attack factor for wave run-rup
if abs(β) <= 20:
    run_up_γβ = 1

elif 20 < abs(β) <= 80:
    run_up_γβ = 1 - 0.0022 * abs(β)

else:
    run_up_γβ = 0.824

    
# General wave run-up height
gen_Ru2 = 1.75 * γf * ξm_1_0 * γb * run_up_γβ

# Max wave run-up height
max_Ru2 = 1 * γf * run_up_γβ * (4.3 - (1.6 / (math.sqrt(ξm_1_0))))

# Relative wave run-up height 
# {minimum of General wave run-up height and max wave run-up height}
rel_Ru2 = min(gen_Ru2, max_Ru2)


# Oblique wave attack factor for overtopping
if abs(β) <= 20:
    ovt_γβ = 1

elif 20 < abs(β) <= 80:
    ovt_γβ = 1 - 0.0033 * abs(β)

else:
    ovt_γβ = 0.736 


# General wave overtopping
gen_q = (0.067/math.sqrt(tanα)) *γb*ξm_1_0* (math.exp(-4.3*RC/(ξm_1_0*Hm0*γb*γf*ovt_γβ*γv)))

# Max wave overtopping
max_q = 0.2*(math.exp(-2.3*RC/(Hm0*γf*ovt_γβ)))

# Relative wave overtopping
# {minimum of General wave overtopping and max wave overtopping}
rel_q = min(gen_q, max_q)

# Wave run-up height (Hm0 * Rel Ru2%)
Ru2 = Hm0 * rel_Ru2

# Overtopping volume per meter (q*sqrt(g*Hm0^3))
q = rel_q * math.sqrt(g * Hm0**3) * 1000

In [47]:
Ru2

1.5228742994320401

In [50]:
q

5.106059889934731

In [56]:
def fb_calculation(func_rc):

    RC = func_rc
    # General wave overtopping
    gen_q = (0.067/math.sqrt(tanα)) *γb*ξm_1_0* (math.exp(-4.3*RC/(ξm_1_0*Hm0*γb*γf*ovt_γβ*γv)))

    # Max wave overtopping
    max_q = 0.2*(math.exp(-2.3*RC/(Hm0*γf*ovt_γβ)))

    # Relative wave overtopping
    # {minimum of General wave overtopping and max wave overtopping}
    rel_q = min(gen_q, max_q)

    # Overtopping volume per meter (q*sqrt(g*Hm0^3))
    q = rel_q * math.sqrt(g * Hm0**3) * 1000
    
    return q
    
def find(func_rc, r):
    return np.abs(r - fb_calculation(func_rc))


fsolve(find, x0=0.1, args=(5.0))

array([0.90424789])

In [63]:
import pandas as pd
import numpy as np
import os
import math
from scipy.optimize import fsolve


# Significant Wave Height [unit: meter]
Hm0 = 3.8

# Peak Wave Period (Associated) [unit: seconds]
Tp = 3.36*math.sqrt(Hm0)

# Slope of front of structure (1:n)
n = 5

# Berm width [unit: meter]
B = 0

# Berm length [unit: meter]
Lberm = 1

# Depth at berm [unit: meter]
db = 0

# Estimate of Ru2% [unit: meter]
Ru2_estimate = 4.2

# Slope roughness factor
γf = 1

# Angle of attack [unit: degree]
β = 20

# Vertical wall factor
γv = 1

# Gravitational acceleration
g = 9.81


# Slope of front of structure {tan(α) = 1/n}
tanα = 1/n

# Spectral wave period {Tm-1,0 = Tp / 1.1}
Tm_1_0 = Tp / 1.1

# Deep water wave length {Lm-1,0=g*Tm-1,0^2/(2*PI)}
Lm_1_0 = (g * Tm_1_0**2) /(2 * math.pi)

# Wave steepness {Hm0/Lm-1,0}
Sm_1_0 = Hm0 / Lm_1_0

# Breaker parameter, Xi {ξm-1,0 = tan(α)/(sm-1,0)**0.5}
# Surf similarity or Iribarren number. Xi<2-3 for breaking waves
ξm_1_0 = tanα / (Sm_1_0**0.5)

# Berm reduction, first part {rB = B/Lberm}
rB = B / Lberm


# Berm reduction, second part
# {db>0: 0.5-0.5*cos(pi*db/Ru2%)  db<0: 0.5-0.5*cos(pi*db/(2*hm0)). Always >=0.6}

if db>0:
    rdb_cal = 0.5 - 0.5 * math.cos(math.pi * db / (Ru2_estimate) )
    rdb = max(0.6, rdb_cal)

elif db<0:
    rdb_cal = 0.5 - 0.5 * math.cos(math.pi * db / (2 * Hm0) )
    rdb = max(0.6, rdb_cal)

else:
    rdb = 0.6

    
# Berm influence factor (Between 0.6 and 1.0)
γb = max(1, min(0.6, 1 - rB * (1 - rdb)))


# Oblique wave attack factor for wave run-rup
if abs(β) <= 20:
    run_up_γβ = 1

elif 20 < abs(β) <= 80:
    run_up_γβ = 1 - 0.0022 * abs(β)

else:
    run_up_γβ = 0.824

# Oblique wave attack factor for overtopping
if abs(β) <= 20:
    ovt_γβ = 1

elif 20 < abs(β) <= 80:
    ovt_γβ = 1 - 0.0033 * abs(β)

else:
    ovt_γβ = 0.736
    

def fb_calculation(func_rc):

    RC = func_rc
    # General wave overtopping
    gen_q = (0.067/math.sqrt(tanα)) *γb*ξm_1_0* (math.exp(-4.3*RC/(ξm_1_0*Hm0*γb*γf*ovt_γβ*γv)))

    # Max wave overtopping
    max_q = 0.2*(math.exp(-2.3*RC/(Hm0*γf*ovt_γβ)))

    # Relative wave overtopping
    # {minimum of General wave overtopping and max wave overtopping}
    rel_q = min(gen_q, max_q)

    # Overtopping volume per meter (q*sqrt(g*Hm0^3))
    q = rel_q * math.sqrt(g * Hm0**3) * 1000
    
    return q
    
def find(func_rc, allowable_q):
    return np.abs(allowable_q - fb_calculation(func_rc))



# CALCULATING FREEBOARD FOR 5 l/s/m OVERTOPPING VOLUME

# Optimization algorithm documentation
#(https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html)
RC = fsolve(find, x0=0.1, args=(5.0))[0]



# CALCULATING RELATIVE WAVE RUNUP HEIGHT

# General wave run-up height
gen_Ru2 = 1.75 * γf * ξm_1_0 * γb * run_up_γβ

# Max wave run-up height
max_Ru2 = 1 * γf * run_up_γβ * (4.3 - (1.6 / (math.sqrt(ξm_1_0))))

# Relative wave run-up height 
# {minimum of General wave run-up height and max wave run-up height}
rel_Ru2 = min(gen_Ru2, max_Ru2)


# CALCULATING OVERTOPPING VOLUME

# General wave overtopping
gen_q = (0.067/math.sqrt(tanα)) *γb*ξm_1_0* (math.exp(-4.3*RC/(ξm_1_0*Hm0*γb*γf*ovt_γβ*γv)))

# Max wave overtopping
max_q = 0.2*(math.exp(-2.3*RC/(Hm0*γf*ovt_γβ)))

# Relative wave overtopping
# {minimum of General wave overtopping and max wave overtopping}
rel_q = min(gen_q, max_q)


# Wave run-up height (Hm0 * Rel Ru2%)
Ru2 = Hm0 * rel_Ru2

# Overtopping volume per meter (q*sqrt(g*Hm0^3))
q = rel_q * math.sqrt(g * Hm0**3) * 1000

In [64]:
RC

4.232431539089653

In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel(r"G:\MJI\SKZ\Script\Freeboard_Calculation_from_Wave_Height\CC_SWH.xlsx")

In [2]:
df

,Return Period,K1,K2,K3,K4,K5,K6,K7,K8
0,10,1.769,2.013,1.944,2.230,2.833,2.718,2.833,2.400
1,20,2.204,2.533,2.509,2.708,3.333,3.178,3.222,2.890
2,25,2.330,2.684,2.677,2.848,3.476,3.310,3.331,3.031
3,30,2.428,2.804,2.811,2.960,3.588,3.413,3.415,3.140
4,35,2.509,2.901,2.921,3.051,3.679,3.497,3.484,3.230
5,40,2.577,2.984,3.015,3.129,3.756,3.568,3.541,3.306
6,45,2.635,3.056,3.096,3.197,3.822,3.629,3.591,3.371
7,50,2.687,3.119,3.168,3.257,3.881,3.683,3.634,3.429
8,55,2.733,3.176,3.233,3.310,3.933,3.731,3.672,3.480
9,60,2.775,3.227,3.291,3.358,3.979,3.774,3.707,3.526


In [3]:
point_df = df.iloc[:,[0,3]]

In [4]:
point_df

,Return Period,K3
0,10,1.944
1,20,2.509
2,25,2.677
3,30,2.811
4,35,2.921
5,40,3.015
6,45,3.096
7,50,3.168
8,55,3.233
9,60,3.291


In [10]:
for point_col_index in range(1, len(df.columns),1):
    point_df = df.iloc[:,[0, point_col_index]]

In [11]:
point_df

,Return Period,K8
0,10,2.400
1,20,2.890
2,25,3.031
3,30,3.140
4,35,3.230
5,40,3.306
6,45,3.371
7,50,3.429
8,55,3.480
9,60,3.526


In [7]:
for i in range(0,len(point_df),1):
    
    row_array = point_df.iloc[i,:].values
    
    return_period = row_array[0]
    sig_wave_height = row_array[1]

In [9]:
sig_wave_height

3.624

In [17]:
import pandas as pd
import numpy as np
import os
import math
from scipy.optimize import fsolve

def fb_calculation(func_rc):

    RC = func_rc
    # General wave overtopping
    gen_q = (0.067/math.sqrt(tanα)) *γb*ξm_1_0* (math.exp(-4.3*RC/(ξm_1_0*Hm0*γb*γf*ovt_γβ*γv)))

    # Max wave overtopping
    max_q = 0.2*(math.exp(-2.3*RC/(Hm0*γf*ovt_γβ)))

    # Relative wave overtopping
    # {minimum of General wave overtopping and max wave overtopping}
    rel_q = min(gen_q, max_q)

    # Overtopping volume per meter (q*sqrt(g*Hm0^3))
    q = rel_q * math.sqrt(g * Hm0**3) * 1000

    return q

def find(func_rc, allowable_q):
    return np.abs(allowable_q - fb_calculation(func_rc))


df = pd.read_excel(r"G:\MJI\SKZ\Script\Freeboard_Calculation_from_Wave_Height\CC_SWH.xlsx")


for point_col_index in range(1, len(df.columns),1):
    point_df = df.iloc[:,[0, point_col_index]]
    
    point_wh_fb_df = pd.DataFrame()
    
    
    for i in range(0,len(point_df),1):
    
        row_array = point_df.iloc[i,:].values

        return_period = row_array[0]
        sig_wave_height = row_array[1]

        # Significant Wave Height [unit: meter]
        Hm0 = sig_wave_height

        # Peak Wave Period (Associated) [unit: seconds]
        Tp = 3.36*math.sqrt(Hm0)

        # Slope of front of structure (1:n)
        n = 5

        # Berm width [unit: meter]
        B = 0

        # Berm length [unit: meter]
        Lberm = 1

        # Depth at berm [unit: meter]
        db = 0

        # Estimate of Ru2% [unit: meter]
        Ru2_estimate = 4.2

        # Slope roughness factor
        γf = 1

        # Angle of attack [unit: degree]
        β = 20

        # Vertical wall factor
        γv = 1

        # Gravitational acceleration
        g = 9.81


        # Slope of front of structure {tan(α) = 1/n}
        tanα = 1/n

        # Spectral wave period {Tm-1,0 = Tp / 1.1}
        Tm_1_0 = Tp / 1.1

        # Deep water wave length {Lm-1,0=g*Tm-1,0^2/(2*PI)}
        Lm_1_0 = (g * Tm_1_0**2) /(2 * math.pi)

        # Wave steepness {Hm0/Lm-1,0}
        Sm_1_0 = Hm0 / Lm_1_0

        # Breaker parameter, Xi {ξm-1,0 = tan(α)/(sm-1,0)**0.5}
        # Surf similarity or Iribarren number. Xi<2-3 for breaking waves
        ξm_1_0 = tanα / (Sm_1_0**0.5)

        # Berm reduction, first part {rB = B/Lberm}
        rB = B / Lberm


        # Berm reduction, second part
        # {db>0: 0.5-0.5*cos(pi*db/Ru2%)  db<0: 0.5-0.5*cos(pi*db/(2*hm0)). Always >=0.6}

        if db>0:
            rdb_cal = 0.5 - 0.5 * math.cos(math.pi * db / (Ru2_estimate) )
            rdb = max(0.6, rdb_cal)

        elif db<0:
            rdb_cal = 0.5 - 0.5 * math.cos(math.pi * db / (2 * Hm0) )
            rdb = max(0.6, rdb_cal)

        else:
            rdb = 0.6


        # Berm influence factor (Between 0.6 and 1.0)
        γb = max(1, min(0.6, 1 - rB * (1 - rdb)))


        # Oblique wave attack factor for wave run-rup
        if abs(β) <= 20:
            run_up_γβ = 1

        elif 20 < abs(β) <= 80:
            run_up_γβ = 1 - 0.0022 * abs(β)

        else:
            run_up_γβ = 0.824
        
        
        # Oblique wave attack factor for overtopping
        if abs(β) <= 20:
            ovt_γβ = 1

        elif 20 < abs(β) <= 80:
            ovt_γβ = 1 - 0.0033 * abs(β)

        else:
            ovt_γβ = 0.736 


        # CALCULATING FREEBOARD FOR 5 l/s/m OVERTOPPING VOLUME

        # Optimization algorithm documentation
        #(https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html)
        RC = fsolve(find, x0=0.1, args=(5.0))[0]



        # CALCULATING RELATIVE WAVE RUNUP HEIGHT

        # General wave run-up height
        gen_Ru2 = 1.75 * γf * ξm_1_0 * γb * run_up_γβ

        # Max wave run-up height
        max_Ru2 = 1 * γf * run_up_γβ * (4.3 - (1.6 / (math.sqrt(ξm_1_0))))

        # Relative wave run-up height 
        # {minimum of General wave run-up height and max wave run-up height}
        rel_Ru2 = min(gen_Ru2, max_Ru2)


        # CALCULATING OVERTOPPING VOLUME

        # General wave overtopping
        gen_q = (0.067/math.sqrt(tanα)) *γb*ξm_1_0* (math.exp(-4.3*RC/(ξm_1_0*Hm0*γb*γf*ovt_γβ*γv)))

        # Max wave overtopping
        max_q = 0.2*(math.exp(-2.3*RC/(Hm0*γf*ovt_γβ)))

        # Relative wave overtopping
        # {minimum of General wave overtopping and max wave overtopping}
        rel_q = min(gen_q, max_q)


        # Wave run-up height (Hm0 * Rel Ru2%)
        Ru2 = Hm0 * rel_Ru2

        # Overtopping volume per meter (q*sqrt(g*Hm0^3))
        q = rel_q * math.sqrt(g * Hm0**3) * 1000

        wh_fb_df = pd.DataFrame({'Return Period': [return_period], 'Wave Height (m)': [sig_wave_height],
                                'Freeboard (m)' : [RC], 'Wave Run-up Height (m)': [Ru2], 'Overtopping (l/s/m)': [q]})


        point_wh_fb_df = pd.concat([point_wh_fb_df, wh_fb_df])
    

In [18]:
point_wh_fb_df

,Return Period,Wave Height (m),Freeboard (m),Wave Run-up Height (m),Overtopping (l/s/m)
0,10.0,2.400,2.379437,3.206051,5.0
0,20.0,2.890,3.008213,3.860620,5.0
0,25.0,3.031,3.193428,4.048975,5.0
0,30.0,3.140,3.337810,4.194584,5.0
0,35.0,3.230,3.457785,4.314811,5.0
0,40.0,3.306,3.559618,4.416335,5.0
0,45.0,3.371,3.647082,4.503166,5.0
0,50.0,3.429,3.725409,4.580646,5.0
0,55.0,3.480,3.794499,4.648774,5.0
0,60.0,3.526,3.856985,4.710223,5.0
